In [18]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds

In [19]:
gdf = gpd.read_file('final_cleaned_gdf.geojson')


In [20]:
pixel_size = 20           # Pixel size - uses meters by defualt (for vector data in ESPG coordinates). Im not sure what the best value is
# I tried pixel_size=5 but I didn't have enough memory to run it on my computer
bounds = gdf.total_bounds # Determine correct bounds for the dataset

# make the bounds consistent with pixel size
transform = from_bounds(*bounds, width=int((bounds[2] - bounds[0]) / pixel_size), height=int((bounds[3] - bounds[1]) / pixel_size)) 


In [21]:
# get the column names in a list
attribute_names = gdf.columns.to_list()
print(attribute_names)

['operator_number', 'flowline_id', 'location_id', 'status', 'location_type', 'fluid', 'material', 'diameter_in', 'length_ft', 'max_operating_pressure', 'shape_length', 'line_age_yr', 'root_cause', 'risk', 'geometry']


In [22]:
# Convert to raster
out_shape = (
    int((bounds[3] - bounds[1]) / pixel_size),  # height in pixels
    int((bounds[2] - bounds[0]) / pixel_size)   # width in pixels
)


attributes = ['operator_number', 'flowline_id', 'location_id', 'status', 'location_type', 'fluid', 
              'material', 'diameter_in', 'length_ft', 'max_operating_pressure', 'shape_length', 
              'line_age_yr', 'root_cause', 'risk']

# create a raster layer for each attribute
rasters = {}
for attribute in attributes:
    shapes = [(geom, value) for geom, value in zip(gdf.geometry, gdf[attribute])]
    raster = rasterize(shapes, out_shape=out_shape, transform=transform, fill=0, dtype='uint8')
    rasters[attribute] = raster  # Store each raster in a dictionary for access later


In [23]:
# Save to file
with rasterio.open(
    "final_cleaned_raster.tif",
    "w",
    driver="GTiff",
    height=raster.shape[0],
    width=raster.shape[1],
    count=1,
    dtype=raster.dtype,
    crs=gdf.crs,
    transform=transform,
) as dst:
    dst.write(raster, 1)